In [75]:
from pase.models.frontend import wf_builder
pase = wf_builder('pase/cfg/PASE.cfg')
pase.eval()
pase.load_pretrained('pase/PASE.ckpt', load_last=True, verbose=True)

# Now we can forward waveforms as Torch tensors
import torch
x = torch.randn(1, 1, 100000)
# y size will be (1, 100, 625), which are 625 frames of 100 dims each
print(x.shape)
y = pase(x)


Current Model keys:  69
Current Pt keys:  69
Loading matching keys:  ['blocks.0.conv.low_hz_', 'blocks.0.conv.band_hz_', 'blocks.0.norm.weight', 'blocks.0.norm.bias', 'blocks.0.norm.running_mean', 'blocks.0.norm.running_var', 'blocks.0.norm.num_batches_tracked', 'blocks.0.act.weight', 'blocks.1.conv.weight', 'blocks.1.conv.bias', 'blocks.1.norm.weight', 'blocks.1.norm.bias', 'blocks.1.norm.running_mean', 'blocks.1.norm.running_var', 'blocks.1.norm.num_batches_tracked', 'blocks.1.act.weight', 'blocks.2.conv.weight', 'blocks.2.conv.bias', 'blocks.2.norm.weight', 'blocks.2.norm.bias', 'blocks.2.norm.running_mean', 'blocks.2.norm.running_var', 'blocks.2.norm.num_batches_tracked', 'blocks.2.act.weight', 'blocks.3.conv.weight', 'blocks.3.conv.bias', 'blocks.3.norm.weight', 'blocks.3.norm.bias', 'blocks.3.norm.running_mean', 'blocks.3.norm.running_var', 'blocks.3.norm.num_batches_tracked', 'blocks.3.act.weight', 'blocks.4.conv.weight', 'blocks.4.conv.bias', 'blocks.4.norm.weight', 'blocks.4.n

In [41]:
import tempfile
import os
import pydub
import scipy
import scipy.io.wavfile


def read_mp3(file_path, as_float = False):
    """
    Read an MP3 File into numpy data.
    :param file_path: String path to a file
    :param as_float: Cast data to float and normalize to [-1, 1]
    :return: Tuple(rate, data), where
        rate is an integer indicating samples/s
        data is an ndarray(n_samples, 2)[int16] if as_float = False
            otherwise ndarray(n_samples, 2)[float] in range [-1, 1]
    """

    path, ext = os.path.splitext(file_path)
    assert ext=='.wav'
    mp3 = pydub.AudioSegment.from_wav(file_path)
    _, path = tempfile.mkstemp()
    mp3.export(path, format="wav")
    rate, data = scipy.io.wavfile.read(path)
    os.remove(path)
    if as_float:
        data = data/(2**15)
    return rate, data

In [56]:
root = '/home/enigmaeth/Videos/accentPhase2/splits'
def get_all_files():
    """
    List all files recursively in the root specified by root
    """
    files_list = []
    dirs = []
    import os

    for dirname, dirnames, filenames in os.walk(root):
        for filename in filenames:
            files_list.append(os.path.join(dirname, filename))

        # Advanced usage:
        # editing the 'dirnames' list will stop os.walk() from recursing into there.
        if '.git' in dirnames:
            # don't go into any .git directories.
            dirnames.remove('.git')
            
    return files_list


In [57]:
files = get_all_files()
files = files[:10]
print(files)

['/home/enigmaeth/Videos/accentPhase2/splits/MSl/MSl_738.wav', '/home/enigmaeth/Videos/accentPhase2/splits/MSl/MSl_273.wav', '/home/enigmaeth/Videos/accentPhase2/splits/MSl/MSl_508.wav', '/home/enigmaeth/Videos/accentPhase2/splits/MSl/MSl_332.wav', '/home/enigmaeth/Videos/accentPhase2/splits/MSl/MSl_629.wav', '/home/enigmaeth/Videos/accentPhase2/splits/MSl/MSl_526.wav', '/home/enigmaeth/Videos/accentPhase2/splits/MSl/MSl_040.wav', '/home/enigmaeth/Videos/accentPhase2/splits/MSl/MSl_631.wav', '/home/enigmaeth/Videos/accentPhase2/splits/MSl/MSl_097.wav', '/home/enigmaeth/Videos/accentPhase2/splits/MSl/MSl_581.wav']


In [78]:
import torch
def run_pipe(n, as_float):
    """
    returns pase vectors for set of files 
    """ 
    files = get_all_files()
    files = files[0:n]
    pase_vectors = []
    for file in files:
        file_rate, file_vec = read_mp3(file, as_float=as_float)
        file_vec_tensor = torch.from_numpy(file_vec)
        file_vec_tensor_flat = torch.flatten(file_vec_tensor)
        file_vec_tensor_view = file_vec_tensor_flat.view(1, 1, file_vec_tensor_flat.shape[0])
        print(file_vec_tensor_view.size())
        file_pase = pase(file_vec_tensor_view)
        pase_vectors.append(file_pase)
        
    return pase_vectors        

In [79]:
print(run_pipe(10, False))

torch.Size([1, 1, 480000])


RuntimeError: "reflection_pad1d" not implemented for 'Short'